# Week04_Report

In [10]:
import pandas as pd
import numpy as np
import math
from scipy.stats import norm, t
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA

## Question 1

In [11]:
n = 100000
sigma = 1
p_t_1 = 100

r_t = np.random.normal(0, sigma, n)

# classical brownian motion
cl_price_t = p_t_1 + r_t
cl_mean = np.mean(cl_price_t)
cl_std_dev = np.std(cl_price_t)

# arithmetic return system
ar_price_t = p_t_1 * (1 + r_t) 
ar_mean = np.mean(ar_price_t)
ar_std_dev = np.std(ar_price_t)

# log return or geometric brownian motion
log_price_t = p_t_1 * np.exp(r_t) 
log_mean = np.mean(log_price_t)
log_std_dev = np.std(log_price_t)

# print simulation results
print("Classical Brownian Motion")
print("mean price at time t", cl_mean)
print("standard deviation of price at time t", cl_std_dev)

print("\nArithmetic Return System")
print("mean price at time t", ar_mean)
print("standard deviation of price at time t", ar_std_dev)

print("\nLog Return or Geometric Brownian Motion")
print("mean price at time t", log_mean)
print("standard deviation of price at time t", log_std_dev)

# calculation by formula with e
result = 100 * math.exp(0.5)
# print(result)

result1 = 100 * math.sqrt((math.exp(1) - 1) * math.exp(1))
# print(result1)


Classical Brownian Motion
mean price at time t 100.00186392342633
standard deviation of price at time t 0.9967223374418334

Arithmetic Return System
mean price at time t 100.18639234263283
standard deviation of price at time t 99.67223374418334

Log Return or Geometric Brownian Motion
mean price at time t 164.5837853637429
standard deviation of price at time t 212.9412992031013


## Question 2

In [16]:
# Read CSV file
prices = pd.read_csv('DailyPrices.csv')

def return_calculate(prices, method="DISCRETE", date_column="Date"):
    
    if date_column not in prices.columns:
        raise ValueError(f"dateColumn: {dateColumn} not in DataFrame: {list(prices.columns)}")
    
    vars = prices.columns
    n_vars = len(vars)
    n_vars = n_vars - 1
    
    # remove date column
    vars = [col for col in prices.columns if col != date_column]
  
    p = prices[vars].values
    n = p.shape[0]
    m = p.shape[1]
    
    # initialize array
    p2 = np.empty((n-1, m))
    
    # calculate returns
    for i in range(n-1):
        for j in range(m):
            p2[i, j] = p[i+1, j] / p[i, j]
    
    # returns based on the specified method
    if method.upper() == "DISCRETE":
        p2 = p2 - 1.0
    elif method.upper() == "LOG":
        p2 = np.log(p2)
    else:
        raise ValueError(f"Unsupported method: {method}")
    
    # extract the date column
    dates = prices[date_column].iloc[1:]
    out = pd.DataFrame({date_column: dates})
    
    # returns to the DataFrame
    for i, var in enumerate(vars):
        out[var] = p2[:, i] 
    
    return out

# calculate arithmetic returns
returns = return_calculate(prices, method="DISCRETE", date_column="Date")
return_mean_meta = returns['META'].mean()
return_removed_meta = returns['META'] - return_mean_meta

print(return_removed_meta.head(10))
# print(return_mean_meta)
# print(returns['META'].std())
# print(return_removed_meta.mean())
# print(return_removed_meta.std())


def calculate_var(results, method="Normal", lambda_ewv=0.94):
    if method == "Normal":
        var = norm.ppf(0.05) * np.std(results)
        
    elif method == "EWV":
        ewv_var = return_removed_meta.ewm(alpha=lambda_ewv).var().iloc[-1]
        ewv_std_dev = np.sqrt(ewv_var)
        var = norm.ppf(0.05) * ewv_std_dev
        
    elif method == "T_DIST":
        params = t.fit(results)
        var = t.ppf(0.05, *params)
        
    elif method == "AR1":
        model_arima = ARIMA(return_removed_meta, order=(1, 0, 0))
        model_arima_fit = model_arima.fit()

        forecast = model_arima_fit.get_forecast(steps=1, alpha=0.05)
        var = norm.ppf(0.05, loc=forecast.predicted_mean.values[-1], scale=forecast.se_mean.values[-1])

    elif method == "HISTORICAL":
        var = np.percentile(results, 5)
        
    else:
        raise ValueError(f"Unsupported method: {method}")

    return var


VaR_normal = calculate_var(return_removed_meta, method="Normal")
VaR_normal_ewv = calculate_var(return_removed_meta, method="EWV")
VaR_t_distn = calculate_var(return_removed_meta, method="T_DIST")
VaR_AR1 = calculate_var(return_removed_meta, method="AR1")
VaR_historical = calculate_var(return_removed_meta, method="HISTORICAL")

print("VaR using normal distribution:", VaR_normal)
print("VaR using normal distribution with exponentially weighted variance:", VaR_normal_ewv )
print("VaR using T distribution:", VaR_t_distn)
print("VaR using a fitted AR(1) model:", VaR_AR1)
print("VaR using historical distribution:", VaR_historical)



1    -0.033266
2    -0.013890
3     0.008882
4     0.007625
5     0.040962
6    -0.003910
7    -0.096478
8    -0.013628
9    -0.015463
10   -0.024586
Name: META, dtype: float64
VaR using normal distribution: -0.054184407435059055
VaR using normal distribution with exponentially weighted variance: -0.028695021689623897
VaR using T distribution: -0.04313471495037609
VaR using a fitted AR(1) model: -0.053730339133240025
VaR using historical distribution: -0.03948424995533789


/var/folders/lf/66gjzvdj5k58dvbny925_tx40000gn/T/ipykernel_49394/2620388888.py:42: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[var] = p2[:, i]
/var/folders/lf/66gjzvdj5k58dvbny925_tx40000gn/T/ipykernel_49394/2620388888.py:42: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  out[var] = p2[:, i]


## Question 3

In [20]:
# Read CSV file
portfolio = pd.read_csv('Portfolio.csv')
# Read CSV file
daily_prices = pd.read_csv('DailyPrices.csv')

# portfolio value
portfolio_values = pd.DataFrame()
portfolio_values['Date'] = daily_prices['Date']

for p in portfolio['Portfolio'].unique():
    portfolio_values[p] = 0
portfolio_values['Total'] = 0

for index, row in portfolio.iterrows():
    stock = row['Stock']
    holding = row['Holding']
    portfolio_type = row['Portfolio']

    if stock in daily_prices.columns:
        daily_value = daily_prices[stock] * holding
        portfolio_values[portfolio_type] += daily_value
portfolio_values['Total'] = portfolio_values[portfolio['Portfolio'].unique()].sum(axis=1)
print("portfolio_values", portfolio_values.head())


# portfolio returns
portfolio_returns = portfolio_values.copy().iloc[:, 1:] 
portfolio_returns = portfolio_returns.pct_change()
portfolio_returns['Date'] = portfolio_values['Date']
cols = portfolio_returns.columns.tolist()
cols = cols[-1:] + cols[:-1]
portfolio_returns_all = portfolio_returns[cols]
portfolio_returns = portfolio_returns_all.dropna().reset_index(drop=True)
portfolio_returns.head()
print("portfolio_returns", portfolio_returns.head())


# portfolio returns exclude mean returns
portfolio_mean_returns = portfolio_returns.iloc[:, 1:].mean()
portfolio_returns_removed = portfolio_returns.iloc[:, 1:].subtract(portfolio_mean_returns, axis=1)
portfolio_returns_removed['Date'] = portfolio_returns['Date']
cols = portfolio_returns_removed.columns.tolist()
cols = cols[-1:] + cols[:-1]
portfolio_returns_removed = portfolio_returns_removed[cols]
print("portfolio_returns_removed",portfolio_returns_removed.head())


# calculate emv VaR
lambda_ewv = 0.94
ewv_var = portfolio_returns_removed.iloc[:, 1:].ewm(alpha=1 - lambda_ewv).var().iloc[-1]
ewv_std_dev = np.sqrt(ewv_var)
VaR_ewv = norm.ppf(0.05) * ewv_std_dev
print("\nVaR_ewv: \n",VaR_ewv)
# calculate dollar VaR based on average portfolio value
average_portfolio_values = portfolio_values.iloc[:, 1:].mean()
average_dollar_var = VaR_ewv * average_portfolio_values
print("\ndollar VaR_ewv based on average portfolio value:\n",average_dollar_var)
# calculate dollar VaR based on latest portfolio value
latest_portfolio_values = portfolio_values.iloc[-1, 1:]
dollar_var_l = VaR_ewv * latest_portfolio_values
print("\ndollar VaR_ewv based on the latest portfolio value:\n", dollar_var_l)



# calculate normal VaR
returns_mean = portfolio_returns_removed.iloc[:, 1:].mean()
returns_std = portfolio_returns_removed.iloc[:, 1:].std()
VaR_normal = norm.ppf(0.05) * returns_std
print("\nVaR_normal: \n",VaR_normal)
# calculate dollar VaR based on average portfolio value
average_portfolio_values_n = portfolio_values.iloc[:, 1:].mean()
average_dollar_var_n = VaR_normal * average_portfolio_values_n
print("\ndollar VaR_normal based on average portfolio value:\n", average_dollar_var_n)
# calculate dollar VaR based on latest portfolio value
latest_portfolio_values_n = portfolio_values.iloc[-1, 1:]
dollar_var_l_n = VaR_normal * latest_portfolio_values_n
print("\ndollar VaR_normal based on the latest portfolio value:\n", dollar_var_l_n)


portfolio_values          Date              A              B             C         Total
0  2022-09-01  864511.243155  524138.141658  1.095803e+06  2.484453e+06
1  2022-09-02  855196.252739  519074.251166  1.081036e+06  2.455307e+06
2  2022-09-06  854111.899156  517844.336430  1.074900e+06  2.446856e+06
3  2022-09-07  868575.869512  527874.300107  1.100873e+06  2.497323e+06
4  2022-09-08  875384.565354  532238.524863  1.134243e+06  2.541866e+06
portfolio_returns          Date         A         B         C     Total
0  2022-09-02 -0.010775 -0.009661 -0.013476 -0.011731
1  2022-09-06 -0.001268 -0.002369 -0.005677 -0.003442
2  2022-09-07  0.016935  0.019369  0.024163  0.020625
3  2022-09-08  0.007839  0.008268  0.030313  0.017837
4  2022-09-09  0.014413  0.016863  0.020010  0.017424
portfolio_returns_removed          Date         A         B         C     Total
0  2022-09-02 -0.011711 -0.010085 -0.014428 -0.012567
1  2022-09-06 -0.002204 -0.002793 -0.006629 -0.004278
2  2022-09-07  0.0159